In [1]:
# --- 1. Cài đặt & Import Thư viện ---
!pip install -q librosa noisereduce tqdm

import os
import glob
import json
import random
import gc
import numpy as np
import librosa
import noisereduce as nr
from tqdm import tqdm
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import soundfile as sf

# --- 2. THIẾT LẬP CẤU HÌNH ---

# !!! THAY ĐỔI ĐƯỜNG DẪN NÀY
# Đường dẫn đến thư mục chứa các thư mục con (asthma, covid, healthy, ...)
DATASET_PATH = "/kaggle/input/raw-audio-dataset/" 

# Thư mục lưu kết quả 
NEW_DATA_DIR = "/kaggle/working/CoughDataNGT_HaveVAL"
os.makedirs(NEW_DATA_DIR, exist_ok=True)

# Lớp (thứ tự này sẽ xác định nhãn số: 0, 1, 2, 3)
CLASSES = ["asthma", "covid", "healthy", "tuberculosis"]
label_map = {name: i for i, name in enumerate(CLASSES)}

# Tỷ lệ chia
TRAIN_RATIO = 0.70
VAL_RATIO = 0.10
TEST_RATIO = 0.20 # (Phải tổng cộng là 1.0)

# Cấu hình Audio & Spectrogram
SR = 22050             # Sample Rate
N_MELS = 128           # Số lượng Mel bins
N_FFT = 2048           # Kích thước cửa sổ FFT
HOP_LENGTH = 512       # Bước nhảy (hop)
MAX_LEN_SECONDS = 5    # Độ dài tối đa (giây) - các file dài hơn sẽ bị cắt
MAX_LEN_SAMPLES = SR * MAX_LEN_SECONDS

# Hình dạng spectrogram kết quả (H, W) = (N_MELS, N_FRAMES)
# N_FRAMES = MAX_LEN_SAMPLES // HOP_LENGTH + 1
N_FRAMES = (MAX_LEN_SAMPLES // HOP_LENGTH) + 1
TARGET_SHAPE = (N_MELS, N_FRAMES) 

RANDOM_SEED = 42

print(f"Sample Rate: {SR}, Max Length: {MAX_LEN_SECONDS}s")
print(f"Target Spectrogram Shape: {TARGET_SHAPE}")

Sample Rate: 22050, Max Length: 5s
Target Spectrogram Shape: (128, 216)


In [2]:
# --- 3. CÁC HÀM TIỀN XỬ LÝ ---

def get_patient_id(filepath, class_name):
    """
    Trích xuất Patient ID từ đường dẫn file dựa trên quy tắc của bạn.
    Đã điều chỉnh cho file .wav.
    """
    filename = os.path.basename(filepath)
    if class_name.lower() in ['asthma', 'covid', 'healthy']:
        # Giả định: "id_... .wav"
        return filename.split('_')[0]
    elif class_name.lower() == 'tuberculosis':
        # Giả định: "id_part1_part2_slice.wav"
        return '_'.join(filename.split('_')[:-1])
    else:
        # Quy tắc dự phòng
        return filename.split('_')[0]

def pad_or_crop_audio(y, max_len_samples):
    """
    Pad (bằng số 0) hoặc Cắt audio array về đúng độ dài.
    """
    if len(y) > max_len_samples:
        # Cắt
        y = y[:max_len_samples]
    elif len(y) < max_len_samples:
        # Pad
        y = np.pad(y, (0, max_len_samples - len(y)), mode='constant')
    return y

def wav_to_spec(filepath, sr, n_mels, n_fft, hop_length, max_len_samples):
    """
    Tải 1 file .wav và chuyển thành Mel Spectrogram đã chuẩn hóa kích thước.
    """
    try:
        y, s = librosa.load(filepath, sr=sr)
        
        # 1. Khử nhiễu (Tùy chọn)
        # y = nr.reduce_noise(y=y, sr=s) 
        
        # 2. Loại bỏ khoảng lặng
        y, _ = librosa.effects.trim(y, top_db=20)
        
        # 3. Pad/Crop audio
        y = pad_or_crop_audio(y, max_len_samples)
        
        # 4. Tạo Mel Spectrogram
        S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length)
        
        # 5. Chuyển sang dB
        S_db = librosa.power_to_db(S, ref=np.max)
        
        # 6. Chuẩn hóa (về 0-1)
        S_db = (S_db - S_db.min()) / (S_db.max() - S_db.min() + 1e-6)
        
        # Đảm bảo đúng shape (có thể lệch 1 frame do làm tròn)
        if S_db.shape[1] > TARGET_SHAPE[1]:
            S_db = S_db[:, :TARGET_SHAPE[1]]
        elif S_db.shape[1] < TARGET_SHAPE[1]:
            S_db = np.pad(S_db, ((0,0), (0, TARGET_SHAPE[1] - S_db.shape[1])), mode='constant')

        return S_db.astype(np.float32)

    except Exception as e:
        print(f"Lỗi khi xử lý file {filepath}: {e}")
        return None

def process_ids_wav(patient_ids, patient_groups, patient_labels, target_shape, **kwargs):
    """
    Tải và xử lý tất cả các tệp cho một nhóm Patient ID (từ .wav).
    """
    X_data = []
    y_data = []
    
    print(f"  Đang xử lý {len(patient_ids)} Patient IDs...")
    for pid in tqdm(patient_ids):
        label = patient_labels[pid]
        for fpath in patient_groups[pid]:
            spec = wav_to_spec(fpath, **kwargs)
            if spec is not None and spec.shape == target_shape:
                X_data.append(spec)
                y_data.append(label)
            elif spec is not None:
                print(f"Bỏ qua file có shape không đúng: {fpath} (shape: {spec.shape})")

    if not X_data:
        print("Cảnh báo: Không có dữ liệu để xử lý trong nhóm ID này.")
        return np.array([]), np.array([])
        
    return np.array(X_data, dtype=np.float32), np.array(y_data, dtype=np.int32)

In [3]:
# --- 4. QUÉT, NHÓM VÀ CHIA DỮ LIỆU ---

all_samples = []
patient_groups = defaultdict(list)
patient_labels = {} # Mỗi patient chỉ có 1 nhãn

print("Đang quét tệp và lấy Patient IDs...")
for class_name in CLASSES:
    folder = os.path.join(DATASET_PATH, class_name)
    if not os.path.exists(folder):
        print(f"Cảnh báo: Không tìm thấy thư mục {folder}")
        continue
    
    # Dùng glob để tìm tất cả file .wav trong thư mục con
    files = glob.glob(os.path.join(folder, "*.wav"))
    print(f"Tìm thấy {len(files)} file .wav trong {class_name}")
    
    for fpath in files:
        patient_id = get_patient_id(fpath, class_name)
        all_samples.append({
            "path": fpath,
            "patient_id": patient_id,
            "label": label_map[class_name]
        })

print(f"\nTổng cộng tìm thấy {len(all_samples)} tệp.")

# Nhóm các tệp theo Patient ID
for sample in all_samples:
    pid = sample["patient_id"]
    patient_groups[pid].append(sample["path"])
    if pid not in patient_labels:
        patient_labels[pid] = sample["label"]

unique_patient_ids = list(patient_groups.keys())
unique_labels = [patient_labels[pid] for pid in unique_patient_ids]

print(f"Tổng cộng có {len(unique_patient_ids)} Patient IDs duy nhất.")
print(f"Phân phối lớp theo Patient ID (Label: Count): {sorted(Counter(unique_labels).items())}")

# --- CHIA DỮ LIỆU THEO NHÓM (PATIENT ID) ---

print("\n--- Bắt đầu chia dữ liệu theo Patient ID ---")
if (TRAIN_RATIO + VAL_RATIO + TEST_RATIO) != 1.0:
    raise ValueError("Tổng tỷ lệ (RATIO) phải bằng 1.0")

# Chia lần 1: Tách (Train + Val) vs Test
# test_size = TEST_RATIO
# temp_size (train+val) = 1.0 - TEST_RATIO
print(f"Chia lần 1 ({(TRAIN_RATIO + VAL_RATIO)*100:.0f}% train+val / {TEST_RATIO*100:.0f}% test)...")
train_val_ids, test_ids, y_train_val_ids, y_test_ids = train_test_split(
    unique_patient_ids, 
    unique_labels, 
    test_size=TEST_RATIO,
    random_state=RANDOM_SEED, 
    stratify=unique_labels
)

# Chia lần 2: Tách Train vs Val (từ bộ train_val_ids)
# Tỷ lệ val_size mới = VAL_RATIO / (1.0 - TEST_RATIO)
val_ratio_new = VAL_RATIO / (TRAIN_RATIO + VAL_RATIO)
print(f"Chia lần 2 (từ bộ train+val -> {val_ratio_new*100:.0f}% val / {(1-val_ratio_new)*100:.0f}% train)...")
train_ids, val_ids, _, _ = train_test_split(
    train_val_ids, 
    y_train_val_ids, 
    test_size=val_ratio_new,
    random_state=RANDOM_SEED,
    stratify=y_train_val_ids
)

print(f"\nĐã chia Patient IDs (Mục tiêu: 70% / 10% / 20%):")
total_ids = len(unique_patient_ids)
print(f"Train IDs: {len(train_ids)} ({len(train_ids)/total_ids:.1%})")
print(f"Val IDs:   {len(val_ids)} ({len(val_ids)/total_ids:.1%})")
print(f"Test IDs:  {len(test_ids)} ({len(test_ids)/total_ids:.1%})")

Đang quét tệp và lấy Patient IDs...
Tìm thấy 3606 file .wav trong asthma
Tìm thấy 43849 file .wav trong covid
Tìm thấy 58811 file .wav trong healthy
Tìm thấy 8637 file .wav trong tuberculosis

Tổng cộng tìm thấy 114903 tệp.
Tổng cộng có 60292 Patient IDs duy nhất.
Phân phối lớp theo Patient ID (Label: Count): [(0, 1809), (1, 21977), (2, 29473), (3, 7033)]

--- Bắt đầu chia dữ liệu theo Patient ID ---
Chia lần 1 (80% train+val / 20% test)...
Chia lần 2 (từ bộ train+val -> 13% val / 88% train)...

Đã chia Patient IDs (Mục tiêu: 70% / 10% / 20%):
Train IDs: 42203 (70.0%)
Val IDs:   6030 (10.0%)
Test IDs:  12059 (20.0%)


In [4]:
# --- 5. XỬ LÝ, LƯU FILE .NPY, VÀ LƯU ẢNH MẪU ---

# Định nghĩa các tham số cho hàm wav_to_spec
spec_params = {
    "sr": SR,
    "n_mels": N_MELS,
    "n_fft": N_FFT,
    "hop_length": HOP_LENGTH,
    "max_len_samples": MAX_LEN_SAMPLES
}

print("\n--- Bắt đầu xử lý và lưu các bộ dữ liệu ---")

print("\nĐang xử lý tập Train...")
X_train, y_train = process_ids_wav(train_ids, patient_groups, patient_labels, TARGET_SHAPE, **spec_params)
print(f"Kích thước X_train: {X_train.shape}, y_train: {y_train.shape}")

print("\nĐang xử lý tập Validation...")
X_val, y_val = process_ids_wav(val_ids, patient_groups, patient_labels, TARGET_SHAPE, **spec_params)
print(f"Kích thước X_val: {X_val.shape}, y_val: {y_val.shape}")

print("\nĐang xử lý tập Test...")
X_test, y_test = process_ids_wav(test_ids, patient_groups, patient_labels, TARGET_SHAPE, **spec_params)
print(f"Kích thước X_test: {X_test.shape}, y_test: {y_test.shape}")

print(f"\n--- Đang lưu các file .npy vào thư mục: {NEW_DATA_DIR} ---")
np.save(os.path.join(NEW_DATA_DIR, "X_train.npy"), X_train)
np.save(os.path.join(NEW_DATA_DIR, "y_train.npy"), y_train)
np.save(os.path.join(NEW_DATA_DIR, "X_val.npy"), X_val)
np.save(os.path.join(NEW_DATA_DIR, "y_val.npy"), y_val)
np.save(os.path.join(NEW_DATA_DIR, "X_test.npy"), X_test)
np.save(os.path.join(NEW_DATA_DIR, "y_test.npy"), y_test)

# Tính và lưu class weights MỚI (chỉ dựa trên tập train MỚI)
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
np.save(os.path.join(NEW_DATA_DIR, "class_weights.npy"), class_weights)
print("Đã lưu xong file class_weights.npy mới.")
print(f"Class weights: {dict(enumerate(class_weights))}")

# Lưu metadata
class_weight_dict = {i: w for i, w in enumerate(class_weights)}
metadata_save = {
    'classes': CLASSES,
    'class_weight_dict': class_weight_dict,
    'audio_config': {
        'sr': SR,
        'n_mels': N_MELS,
        'n_fft': N_FFT,
        'hop_length': HOP_LENGTH,
        'max_len_seconds': MAX_LEN_SECONDS
    },
    'target_shape': TARGET_SHAPE,
    'train_samples': len(X_train),
    'val_samples': len(X_val),
    'test_samples': len(X_test),
    'train_ids': len(train_ids),
    'val_ids': len(val_ids),
    'test_ids': len(test_ids)
}
with open(os.path.join(NEW_DATA_DIR, "metadata.json"), 'w') as f:
    json.dump(metadata_save, f, indent=2)
print("Đã lưu xong file metadata.json mới.")

# --- 6. (THÊM MỚI) LƯU ẢNH MẪU ---
print("\n--- Đang lưu 3 ảnh mẫu cho mỗi lớp ---")
img_save_path = os.path.join(NEW_DATA_DIR, "sample_images")
os.makedirs(img_save_path, exist_ok=True)
N_SAMPLES_TO_SAVE = 3

for class_idx, class_name in enumerate(CLASSES):
    # Lấy chỉ mục của các mẫu thuộc lớp này (từ tập train)
    class_indices = np.where(y_train == class_idx)[0]
    
    if len(class_indices) == 0:
        print(f"Cảnh báo: Không tìm thấy mẫu nào cho lớp '{class_name}' trong tập train.")
        continue
        
    # Chọn ngẫu nhiên N mẫu
    sample_indices = np.random.choice(class_indices, 
                                      size=min(N_SAMPLES_TO_SAVE, len(class_indices)), 
                                      replace=False)
    
    print(f"  Lưu mẫu cho lớp: {class_name}")
    for i, sample_idx in enumerate(sample_indices):
        spec = X_train[sample_idx]
        
        plt.figure(figsize=(10, 4))
        librosa.display.specshow(spec, sr=SR, hop_length=HOP_LENGTH, x_axis='time', y_axis='mel', cmap='viridis')
        plt.title(f"{class_name} - Sample {i+1}")
        plt.colorbar(format='%+2.0f dB')
        plt.tight_layout()
        
        # Lưu file
        save_name = os.path.join(img_save_path, f"{class_name}_sample_{i+1}.png")
        plt.savefig(save_name)
        plt.close()

print(f"\n✅ Hoàn tất! Dữ liệu đã sẵn sàng trong thư mục: '{NEW_DATA_DIR}'")


--- Bắt đầu xử lý và lưu các bộ dữ liệu ---

Đang xử lý tập Train...
  Đang xử lý 42203 Patient IDs...


  0%|          | 89/42203 [00:18<37:55, 18.51it/s]/tmp/ipykernel_19/640039733.py:36: UserWarning: PySoundFile failed. Trying audioread instead.
  y, s = librosa.load(filepath, sr=sr)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
  0%|          | 91/42203 [00:18<37:47, 18.57it/s]

Lỗi khi xử lý file /kaggle/input/raw-audio-dataset/healthy/P0000594_71-14-8f870715-6afd-49ed-8c0c6980c1da2f59.wav: 


  0%|          | 112/42203 [00:19<34:34, 20.29it/s]

Lỗi khi xử lý file /kaggle/input/raw-audio-dataset/healthy/P0067855_71-14-96625019-c592-4670-ab96fde1fd482e88.wav: 
Lỗi khi xử lý file /kaggle/input/raw-audio-dataset/covid/P0000653_62-13-000c032c-7baa-466f-b3aafc2dfa00d9f9.wav: 
Lỗi khi xử lý file /kaggle/input/raw-audio-dataset/covid/P0000653_62-14-b8e2ccfa-23ba-4953-973633ea2d400f4f.wav: 


...

Lỗi khi xử lý file /kaggle/input/raw-audio-dataset/healthy/P0066850_636_sfi5E7jqGX0ZDe4M.wav: 
Lỗi khi xử lý file /kaggle/input/raw-audio-dataset/healthy/P0066850_636_ba2UrtDhh2chxaM0.wav: 


 99%|█████████▉| 11939/12059 [11:04<00:06, 17.69it/s]

Lỗi khi xử lý file /kaggle/input/raw-audio-dataset/covid/P0067708_62-14-e3b1a42d-e3b2-4a18-b3f4146dc15d2bcc.wav: 


100%|██████████| 12059/12059 [11:10<00:00, 17.98it/s]


Kích thước X_test: (22849, 128, 216), y_test: (22849,)

--- Đang lưu các file .npy vào thư mục: /kaggle/working/CoughDataNGT_HaveVAL ---
Đã lưu xong file class_weights.npy mới.
Class weights: {0: 7.9552327894946275, 1: 0.6554805075576249, 2: 0.4885985922377554, 3: 3.310947333554157}
Đã lưu xong file metadata.json mới.

--- Đang lưu 3 ảnh mẫu cho mỗi lớp ---
  Lưu mẫu cho lớp: asthma
  Lưu mẫu cho lớp: covid
  Lưu mẫu cho lớp: healthy
  Lưu mẫu cho lớp: tuberculosis

✅ Hoàn tất! Dữ liệu đã sẵn sàng trong thư mục: '/kaggle/working/CoughDataNGT_HaveVAL'
